In [9]:
from getpass import getpass
from sqlalchemy import create_engine
import pandas as pd

from surprise import Dataset
from surprise import Reader

In [2]:
# MySQL password
pw = getpass()

········


In [3]:
cnx = create_engine('mysql+pymysql://'+'root'+':'+pw+'@'+'localhost'+':3306/'+'yelp',echo=False)

In [4]:
restaurants = pd.read_sql('business_rest', cnx)

/Users/edgarpal/anaconda3/lib/python3.6/site-packages/pymysql/cursors.py:170: Warning: (3719, "'utf8' is currently an alias for the character set UTF8MB3, but will be an alias for UTF8MB4 in a future release. Please consider using UTF8MB4 in order to be unambiguous.")
  result = self._query(query)


In [5]:
review = pd.read_sql('select review_id, user_id, business_id, stars from review', cnx)

In [6]:
reviews_rest = review.merge(restaurants, how = 'inner', on = 'business_id', suffixes = ('_rev', '_rest'))

In [10]:
reviews_rest.shape

(3655032, 16)

In [7]:
# Percentage of reviews that are for restaurants
float(reviews_rest.shape[0] / review.shape[0])

0.6094771448905418

In [8]:
reviews_rest.head(5)

,review_id,user_id,business_id,stars_rev,address,categories,city,is_open,latitude,longitude,name,neighborhood,postal_code,review_count,stars_rest,state
0,x7mDIiDB3jEiPGPHOmDzyw,msQe1u7Z_XuqjGoqhB0J5g,iCQpiavjjPzJ5_3gPD5Ebg,2,3708 Las Vegas Blvd S,"Pizza, Restaurants",Las Vegas,1,36.109837,-115.174212,Secret Pizza,The Strip,89109,4078,4.0,NV
1,VluIpojME0yKOcRKI5L0Iw,D1_nrBr4dOrs7M82OaBRwQ,iCQpiavjjPzJ5_3gPD5Ebg,3,3708 Las Vegas Blvd S,"Pizza, Restaurants",Las Vegas,1,36.109837,-115.174212,Secret Pizza,The Strip,89109,4078,4.0,NV
2,nsB9JAeghk0zOaSulSm9Yw,jEjc8HxeCeI22M4di0Y1aw,iCQpiavjjPzJ5_3gPD5Ebg,5,3708 Las Vegas Blvd S,"Pizza, Restaurants",Las Vegas,1,36.109837,-115.174212,Secret Pizza,The Strip,89109,4078,4.0,NV
3,pcn01EMERpCpHEcoaohdEg,RTiaaHSN80VRTLSdNK5rfQ,iCQpiavjjPzJ5_3gPD5Ebg,5,3708 Las Vegas Blvd S,"Pizza, Restaurants",Las Vegas,1,36.109837,-115.174212,Secret Pizza,The Strip,89109,4078,4.0,NV
4,0mFtAPTmInbXHqDjX9eiOg,hBmJs49zYcXdV4wo8F5iiA,iCQpiavjjPzJ5_3gPD5Ebg,5,3708 Las Vegas Blvd S,"Pizza, Restaurants",Las Vegas,1,36.109837,-115.174212,Secret Pizza,The Strip,89109,4078,4.0,NV


In [11]:
# A reader is still needed but only the rating_scale param is requiered.
reader = Reader(rating_scale=(1, 5))

In [12]:
# The columns must correspond to user id, item id and ratings (in that order).
data = Dataset.load_from_df(reviews_rest[['user_id', 'business_id', 'stars_rev']], reader)